In [34]:
from tqdm.notebook import tqdm
tqdm.pandas()

import pandas as pd
import numpy as np

from sklearn.metrics import cohen_kappa_score
from statsmodels.stats.inter_rater import fleiss_kappa

# Fix the random seed for reproducibility
np.random.seed(42)

In [35]:
# Data generation
ids = [f'ID_{i}' for i in range(1, 51)]
random_ids = np.random.choice(ids, 300)
data = {'ID': random_ids}
data.update({f'confirm_{i+1}': np.random.randint(0, 2, 300) for i in range(6)})
df = pd.DataFrame(data)

# Uncomment to use your own dataset
# df = pd.read_excel("your_dataset.xlsx")

In [36]:
df.head(3)

,ID,confirm_1,confirm_2,confirm_3,confirm_4,confirm_5,confirm_6
0,ID_39,1,0,0,0,0,1
1,ID_29,1,1,1,1,1,0
2,ID_15,1,1,0,1,0,0


In [37]:
# Check if specified columns exist in DataFrame
def check_columns_exist(df, columns):
    return all(col in df.columns for col in columns)

# Calculate Cohen's Kappa; Ensure necessary columns exist
def calculate_cohen_kappa(df, rater_columns):
    if not check_columns_exist(df, rater_columns):
        return "Missing columns in DataFrame."

    return {f'{r1}_vs_{r2}': cohen_kappa_score(df[r1], df[r2])
            for i, r1 in enumerate(rater_columns)
            for j, r2 in enumerate(rater_columns) if i < j}

# Calculate Fleiss Kappa
def Fleiss_kappa(df):
    # Count the number of each score (0 or 1) for each row, fill missing values with 0
    value_counts = df.progress_apply(pd.value_counts, axis=1).fillna(0)
    # Compute Fleiss' Kappa using the counts
    return fleiss_kappa(value_counts.to_numpy())

# Calculate Kappa for each unique ID
def calculate_kappa_per_id(df, rater_columns):
    if not check_columns_exist(df, ['ID'] + rater_columns):
        return "Missing columns in DataFrame."

    grouped = df.groupby("ID")
    kappa_scores = []

    for name, group in tqdm(grouped):
        if len(group) < 2: # Skip groups with fewer than 2 rows
            continue

        kappa_values = {'ID': name, 'Fleiss Kappa': Fleiss_kappa(group[rater_columns])}

        for i, r1 in enumerate(rater_columns):
            for j, r2 in enumerate(rater_columns):
                if i >= j:
                    continue
                kappa = cohen_kappa_score(group[r1], group[r2]) if len(set(group[r1])) > 1 and len(set(group[r2])) > 1 else 'N/A'
                kappa_values[f'{r1}_vs_{r2}'] = kappa

        kappa_scores.append(kappa_values)

    return pd.DataFrame(kappa_scores)


In [38]:
# Dynamically generate rater column names
rater_columns = [col for col in df.columns if 'confirm_' in col]
cohen_kappa_results = calculate_cohen_kappa(df, rater_columns)
each_kappa = calculate_kappa_per_id(df, rater_columns)

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

In [39]:
rater_columns

['confirm_1', 'confirm_2', 'confirm_3', 'confirm_4', 'confirm_5', 'confirm_6']

In [40]:
cohen_kappa_results

{'confirm_1_vs_confirm_2': 0.06498673740053063,
 'confirm_1_vs_confirm_3': 0.001322751322751281,
 'confirm_1_vs_confirm_4': 0.04539385847797073,
 'confirm_1_vs_confirm_5': 0.019218025182239917,
 'confirm_1_vs_confirm_6': 0.041005291005290934,
 'confirm_2_vs_confirm_3': 0.00910551687198724,
 'confirm_2_vs_confirm_4': 0.12062533309646462,
 'confirm_2_vs_confirm_5': -0.06340288924558579,
 'confirm_2_vs_confirm_6': 0.03588644884841996,
 'confirm_3_vs_confirm_4': -0.06553010122535974,
 'confirm_3_vs_confirm_5': -0.024919614147909996,
 'confirm_3_vs_confirm_6': -0.03327965646806241,
 'confirm_4_vs_confirm_5': 0.02078337330135893,
 'confirm_4_vs_confirm_6': 0.027703782631859353,
 'confirm_5_vs_confirm_6': -0.07851018220793149}

In [41]:
# Calculate overall Fleiss Kappa
if check_columns_exist(df, rater_columns):
    overall_fleiss_kappa = Fleiss_kappa(df[rater_columns])
else:
    overall_fleiss_kappa = "Missing columns in DataFrame."

print(f"Overall Fleiss Kappa: {overall_fleiss_kappa}")


  0%|          | 0/300 [00:00<?, ?it/s]

Overall Fleiss Kappa: 0.007294075787226483


In [42]:
each_kappa.head(5)

,ID,Fleiss Kappa,confirm_1_vs_confirm_2,confirm_1_vs_confirm_3,confirm_1_vs_confirm_4,confirm_1_vs_confirm_5,confirm_1_vs_confirm_6,confirm_2_vs_confirm_3,confirm_2_vs_confirm_4,confirm_2_vs_confirm_5,confirm_2_vs_confirm_6,confirm_3_vs_confirm_4,confirm_3_vs_confirm_5,confirm_3_vs_confirm_6,confirm_4_vs_confirm_5,confirm_4_vs_confirm_6,confirm_5_vs_confirm_6
0,ID_1,0.014957,0.384615,0.04,0.25,0.157895,-0.263158,0.157895,0.75,0.058824,-0.411765,0.25,0.384615,-0.230769,0.25,-0.25,-0.6
1,ID_11,-0.051826,-0.230769,-0.263158,0.04,-0.230769,-0.25,0.058824,0.157895,-0.066667,0.25,-0.230769,0.058824,0.25,-0.263158,0.25,0.25
2,ID_12,-0.028571,-0.5,0.2,-0.5,-0.5,N/A,0.5,0.0,1.0,N/A,-0.5,0.5,N/A,0.0,N/A,N/A
3,ID_13,-0.163077,N/A,0.4,N/A,0.4,-0.8,N/A,N/A,N/A,N/A,N/A,-0.5,-0.5,N/A,N/A,-0.5
4,ID_14,-0.136842,0.181818,0.0,0.571429,-0.5,-0.2,0.333333,0.076923,-0.363636,-0.285714,0.333333,-0.666667,-0.666667,-0.285714,-0.363636,0.4
